In [83]:
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
from pathlib import Path

import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# ==========================================================
# CONFIGURATION
# ==========================================================
np.random.seed(42)
torch.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 Device: {device}")

# Paths
ROOT = Path("Amazon_products")
TRAIN_CORPUS_PATH = ROOT / "train" / "train_corpus.txt"
TEST_CORPUS_PATH  = ROOT / "test" / "test_corpus.txt"
CLASS_PATH        = ROOT / "classes.txt"
CLASS_HIERARCHY_PATH = ROOT / "class_hierarchy.txt" 

EMB_DIR          = Path("Embeddings")
X_ALL_PATH       = EMB_DIR / "X_train_test_mpn.pt"
LABEL_EMB_PATH   = EMB_DIR / "labels_hierarchical_new_mpn.pt"

MODEL_SAVE = Path("Models")
MODEL_SAVE.mkdir(exist_ok=True)
MODEL_PATH = MODEL_SAVE / "silver_classifier.pt"

# ==========================================================
# LOAD IDS
# ==========================================================
def load_ids(path):
    ids = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            pid, _ = line.strip().split("\t", 1)
            ids.append(int(pid))
    return ids

train_ids = load_ids(TRAIN_CORPUS_PATH)
test_ids  = load_ids(TEST_CORPUS_PATH)
n_train = len(train_ids)
n_test  = len(test_ids)

print(f"Train IDs: {n_train} | Test IDs: {n_test}")

def load_multilabel(path):
    """Load multi-label data into {id: [labels]} dictionary."""
    id2labels = {}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) == 2:
                pid, label = parts
                pid = int(pid)
                label = int(label)

                if pid not in id2labels:
                    id2labels[pid] = []

                id2labels[pid].append(label)
    return id2labels

# ==========================================================
# LOAD X_all
# ==========================================================
print("\n🧠 Loading X_all.pt ...")
data = torch.load(X_ALL_PATH, weights_only=False)

if isinstance(data, np.ndarray):
    data = torch.from_numpy(data)
elif isinstance(data, list):
    data = torch.stack(data)

X_all = data.float().to(device)
assert X_all.shape[0] == n_train + n_test

X_train = X_all[:n_train]
X_test  = X_all[n_train:]
print(f"✓ X_train: {X_train.shape} | X_test: {X_test.shape}")

# ==========================================================
# LOAD LABEL EMBEDDINGS
# ==========================================================
tmp = torch.load(LABEL_EMB_PATH, weights_only=False)

# Convertir numpy → tensor si nécessaire
if isinstance(tmp, np.ndarray):
    tmp = torch.from_numpy(tmp)

label_emb = tmp.float().to(device)
print(f"✓ Label embeddings: {label_emb.shape}")

# ==========================================================
# LOAD CLASS NAMES
# ==========================================================
classes = {}
with open(CLASS_PATH, "r", encoding="utf-8") as f:
    for line in f:
        cid, cname = line.strip().split("\t")
        classes[int(cid)] = cname

n_classes = len(classes)

pid2idx = {pid: i for i, pid in enumerate(train_ids)}

class2hierarchy = load_multilabel(CLASS_HIERARCHY_PATH)
print(class2hierarchy)



🔧 Device: cuda
Train IDs: 29487 | Test IDs: 19658

🧠 Loading X_all.pt ...
✓ X_train: torch.Size([29487, 768]) | X_test: torch.Size([19658, 768])
✓ Label embeddings: torch.Size([531, 768])
{0: [1, 8, 208, 211, 213, 216, 229, 255, 265, 218, 271, 277, 249, 288, 313, 357], 1: [2, 434, 413, 483, 464, 520, 527], 3: [4, 5, 13, 15, 17, 21, 28, 30, 34, 35, 50, 51, 53, 85, 111, 120, 147], 4: [7, 19, 70, 174, 176, 183, 188, 210, 238, 294, 375, 392, 407, 433, 435], 5: [6, 57, 97, 133, 228, 331, 385], 8: [9, 206, 235, 270, 285, 308, 327, 391, 405, 410, 514], 10: [11, 44, 54, 60, 64, 220], 11: [12, 69, 45, 109, 205, 67, 419], 13: [14, 20, 37, 122, 150, 204, 378, 401, 477, 506, 517], 15: [16, 56, 144, 166], 17: [18, 56, 58, 168, 320], 21: [22, 33, 48, 102, 108, 115, 124, 126, 127, 154, 164, 172, 182, 187, 234, 301, 305, 379, 500], 23: [24, 26, 31, 38, 74, 76, 105], 24: [25, 43, 47, 49, 59, 62, 253, 71, 345], 26: [27, 89, 231, 350], 28: [29, 113, 190, 207, 310, 330, 364, 518], 30: [66, 81, 104, 117, 1

In [84]:
import json

# Load JSON
with open("Silver/silver_train_new_mpn.json", "r", encoding="utf-8") as f:
    raw = json.load(f)

confidence_threshold = 0.62

pid2labelids_silver_filtered = {}

for pid_str, data in raw.items():
    pid = int(pid_str)

    labels = data["labels"]
    probs = data["probs"]

    # Si AU MOINS un score dépasse le seuil → on garde TOUTE la liste
    if any(score > confidence_threshold for score in probs):
        pid2labelids_silver_filtered[pid] = labels

print(f"Filtered: {len(pid2labelids_silver_filtered)} / {len(raw)}")

silver_labels = pid2labelids_silver_filtered

print(silver_labels)


Filtered: 11080 / 29487
{0: [118, 23, 74], 1: [456, 271, 0], 2: [291, 271, 0], 3: [93, 64, 10], 4: [271, 0], 6: [301, 21, 3], 8: [128, 40], 10: [429, 271, 0], 14: [42, 41, 40], 16: [301, 3, 21], 17: [402, 145, 40], 20: [164, 21, 3], 21: [195, 128, 40], 24: [118, 74, 23], 26: [301, 3, 21], 27: [182, 3, 21], 32: [195, 128, 40], 33: [231, 26, 23], 34: [293, 85, 3], 35: [224, 41, 40], 38: [435, 4, 3], 42: [242, 220, 10], 43: [206, 8, 0], 45: [80, 64, 10], 51: [301, 3, 21], 52: [45, 44, 145], 53: [349, 265, 0], 59: [490, 265, 0], 63: [93, 64, 10], 67: [340, 265, 0], 70: [350, 26, 23], 71: [523, 3, 53], 76: [107, 23, 38], 78: [301, 3, 21], 84: [52, 3, 51], 90: [93, 64, 10], 93: [286, 41, 40], 95: [340, 265, 0], 96: [93, 64, 10], 101: [350, 26, 23], 102: [206, 8, 0], 104: [186, 128, 40], 105: [397, 0, 313], 106: [129, 128, 40], 110: [291, 271, 0], 115: [45, 44, 145], 116: [355, 271, 0], 123: [236, 135, 40], 128: [220, 10], 130: [206, 8, 0], 131: [89, 26, 23], 134: [171, 170, 199], 136: [220, 

In [85]:
class MultiLabelDataset(Dataset):
    def __init__(self, pids, labels_dict):
        self.pids = pids
        self.labels = labels_dict

    def __len__(self):
        return len(self.pids)

    def __getitem__(self, idx):
        pid = self.pids[idx]
        emb = X_train[pid2idx[pid]]

        y = torch.zeros(n_classes)
        for c in self.labels[pid]:
            if 0 <= c < n_classes:
                y[c] = 1.0

        return {"X": emb, "y": y}
    

class UnlabeledEmbeddingDataset(Dataset):
    def __init__(self, pids, pid2idx, embeddings):
        self.pids = pids                
        self.pid2idx = pid2idx             
        self.embeddings = embeddings      

    def __len__(self):
        return len(self.pids)

    def __getitem__(self, idx):
        pid = self.pids[idx]
        emb = self.embeddings[self.pid2idx[pid]]

        return {"X": emb, "pid": pid}



In [86]:
train_p, val_p = train_test_split(
    list(silver_labels.keys()), test_size=0.2, random_state=42
)

train_dataset = MultiLabelDataset(train_p, silver_labels)
val_dataset   = MultiLabelDataset(val_p, silver_labels)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=64)

In [87]:
class InnerProductClassifier(nn.Module):
    def __init__(self, input_dim, label_embeddings, dropout=0.2, trainable_label_emb=False):
        super().__init__()

        D = label_embeddings.size(1)

        self.proj = nn.Linear(input_dim, D)
        self.dropout = nn.Dropout(dropout)

        if trainable_label_emb:
            self.label_emb = nn.Parameter(label_embeddings.clone())
        else:
            self.register_buffer("label_emb", label_embeddings.clone())

    def forward(self, x, use_dropout=True):
        if use_dropout:
            x = self.dropout(x)

        x_proj = self.proj(x)              
        logits = x_proj @ self.label_emb.T   

        return logits

In [88]:
def evaluate(model, loader, thr=0.25):
    model.eval()
    preds, labels = [], []

    with torch.no_grad():
        for batch in loader:
            X = batch["X"]
            y = batch["y"].numpy()

            prob = torch.sigmoid(model(X)).cpu().numpy()
            pred = (prob > thr).astype(int)

            preds.extend(pred)
            labels.extend(y)

    f1s = f1_score(labels, preds, average="samples")
    f1m = f1_score(labels, preds, average="macro")
    f1mic = f1_score(labels, preds, average="micro")
    return f1s, f1m, f1mic

In [89]:
# Phase 1 : Pre Training

import copy

print("\nTraining...")
best = 0
epochs = 20
wait = 0
patience = 10

model = InnerProductClassifier(
    input_dim = X_train.size(1),
    label_embeddings = label_emb,
    dropout = 0.2,
    trainable_label_emb = False
).to(device)

best_model = copy.deepcopy(model.state_dict())

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)

for epoch in range(1, epochs+1):
    model.train()
    total = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        X = batch["X"].to(device)
        y = batch["y"].to(device)

        logits = model(X)
        loss = F.binary_cross_entropy_with_logits(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += loss.item()

    f1s, f1m, f1mic = evaluate(model, val_loader)
    print(f"[Epoch {epoch}] loss={total/len(train_loader):.4f} | F1={f1s:.4f}")

    if f1s > best:
        best = f1s
        wait = 0
        best_model = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), MODEL_PATH)
        print(f"New best model saved ({best:.4f})")
    else:
        wait += 1
    
    if wait >= patience:
        print("\nEarly stopping triggered!")
        break


print(f"\nBest validation F1 = {best:.4f}")
print(f"Model saved at: {MODEL_PATH}")


# === Load best student before test ===
model.load_state_dict(best_model)


Training...


Epoch 1: 100%|██████████| 139/139 [00:00<00:00, 174.15it/s]


[Epoch 1] loss=0.1478 | F1=0.0005
New best model saved (0.0005)


Epoch 2: 100%|██████████| 139/139 [00:00<00:00, 250.21it/s]


[Epoch 2] loss=0.0448 | F1=0.0000


...

Epoch 19: 100%|██████████| 139/139 [00:00<00:00, 316.85it/s]


[Epoch 19] loss=0.0175 | F1=0.4272
New best model saved (0.4272)


Epoch 20: 100%|██████████| 139/139 [00:00<00:00, 322.48it/s]


[Epoch 20] loss=0.0171 | F1=0.4420
New best model saved (0.4420)

Best validation F1 = 0.4420
Model saved at: Models\silver_classifier.pt


<All keys matched successfully>

In [90]:
student = model
teacher = InnerProductClassifier(
    input_dim=X_train.size(1),
    label_embeddings=label_emb,
    dropout=0.0,               
    trainable_label_emb=False
).to(device)

teacher.load_state_dict(best_model) 
teacher.eval()

def ema_update(teacher, student, alpha=0.999):
    for t_param, s_param in zip(teacher.parameters(), student.parameters()):
        t_param.data.mul_(alpha).add_(s_param.data * (1 - alpha))


EPOCHS = 100
pseudo_update_freq = 3

threshold_start = 0.40
threshold_end   = 0.65

alpha_ema = 0.995
lambda_cons = 1.5

current_labels = dict(silver_labels) 
all_train_pids = list(pid2idx.keys())

optimizer = torch.optim.AdamW(student.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

wait = 0
patience = 5
best_f1 = 0.
best_teacher = copy.deepcopy(teacher.state_dict())

def consistency_loss(log_s, log_t):
    ps = torch.sigmoid(log_s)
    pt = torch.sigmoid(log_t)
    return F.mse_loss(ps, pt)


In [91]:
def expand_with_hierarchy(labels, hierarchy):
    """
    Expand a list of core labels by adding ALL their ancestors
    (parents, parents of parents, etc.), recursively.
    This guarantees 100% hierarchy consistency.
    """
    expanded = set(labels)
    stack = list(labels)
    child2parents = {}
    for parent, children in hierarchy.items():
        for child in children:
            child2parents.setdefault(child, []).append(parent)

    # DFS / BFS upward through ancestors
    while stack:
        node = stack.pop()

        if node not in child2parents:
            continue

        for parent in child2parents[node]:
            if parent not in expanded:
                expanded.add(parent)
                stack.append(parent)  

    return sorted(expanded)[-3:]


def select_labels_hierarchical(probs, threshold, hierarchy, max_k=3):
    # 1) labels surpassant le seuil
    cand = [i for i in range(len(probs)) if probs[i] > threshold]

    if len(cand) == 0:
        return []

    # ordonner par probas
    cand = sorted(cand, key=lambda c: -probs[c])
    # prendre au maximum 3 feuilles candidates
    cand = cand[:max_k]

    # 2) expansion hiérarchique
    expanded = set(cand)
    for c in cand:
        if str(c) in hierarchy:
            parents = hierarchy[str(c)]
            for p in parents:
                expanded.add(p)

    expanded = list(expanded)

    # 3) garder max 3 labels au total
    expanded = sorted(expanded, key=lambda c: -probs[c])[:max_k]

    return expanded


def invert_hierarchy(class2hierarchy):
    inv = {}
    for parent, children in class2hierarchy.items():
        for child in children:
            inv.setdefault(child, []).append(parent)
    return inv

inv_hierarchy = invert_hierarchy(class2hierarchy)


def generate_pseudo_labels(threshold):
    teacher.eval()
    new_pseudo = {}

    labeled = set(current_labels.keys())
    unlabeled_pids = [pid for pid in all_train_pids if pid not in labeled]

    unlabeled_ds = UnlabeledEmbeddingDataset(unlabeled_pids, pid2idx, X_train)
    unlabeled_ld = DataLoader(unlabeled_ds, batch_size=64)

    with torch.no_grad():
        for batch in unlabeled_ld:
            X = batch["X"].to(device)
            pids = batch["pid"]

            logits = teacher(X)
            probs_batch = torch.sigmoid(logits).cpu().numpy()

            for pid, p in zip(pids, probs_batch):

                final_labels = select_labels_hierarchical(
                    probs=p,
                    threshold=threshold,
                    hierarchy=inv_hierarchy,
                    max_k=3
                )

                if len(final_labels) < 2:
                    continue

                new_pseudo[int(pid)] = final_labels

    return new_pseudo


In [92]:
total = 0

for epoch in range(1, EPOCHS + 1):

    if epoch % pseudo_update_freq == 1 and epoch > 1:
        progress = epoch / EPOCHS
        thr = threshold_start + (threshold_end - threshold_start) * (epoch / EPOCHS)**2

        new_pseudo = generate_pseudo_labels(thr)

        if len(new_pseudo) > 0:
            before = len(current_labels)
            current_labels.update(new_pseudo)
            print(f"Added {len(current_labels) - before} pseudo-labeled examples")
            total += len(current_labels) - before

    train_ds = MultiLabelDataset(list(current_labels.keys()), current_labels)
    train_ld = DataLoader(train_ds, batch_size=64, shuffle=True)

    student.train()
    teacher.eval()

    epoch_loss = 0.

    for batch in train_ld:
        X = batch["X"].to(device)
        y = batch["y"].to(device)

        # student (noisy)
        noise = torch.randn_like(X) * 0.05
        logits_s = student(X + noise)

        # teacher (clean)
        with torch.no_grad():
            logits_t = teacher(X)

        # Multi-label supervised loss
        loss_sup = F.binary_cross_entropy_with_logits(logits_s, y)

        # Consistency loss
        loss_cons = consistency_loss(logits_s, logits_t)
        loss = loss_sup + lambda_cons * loss_cons

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # EMA teacher update
        ema_update(teacher, student, alpha_ema)

        epoch_loss += loss.item()

    scheduler.step()

    # Validation with teacher
    teacher.eval()
    f1s, f1m, f1mic = evaluate(teacher, val_loader)

    print(f"Epoch {epoch} | Loss={epoch_loss/len(train_ld):.4f} | F1={f1s:.4f}")

    if f1s > best_f1:
        best_f1 = f1s
        best_teacher = copy.deepcopy(teacher.state_dict())
        wait = 0
        print(f"New best F1 = {best_f1:.4f}")
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered")
            print(f"Number of PS added {total}")
            break

# Load best teacher
teacher.load_state_dict(best_teacher)
print("\nFinal teacher F1:", best_f1)

Epoch 1 | Loss=0.0190 | F1=0.4304
New best F1 = 0.4304
Epoch 2 | Loss=0.0170 | F1=0.4623
New best F1 = 0.4623
Epoch 3 | Loss=0.0157 | F1=0.5026
New best F1 = 0.5026
Added 989 pseudo-labeled examples
Epoch 4 | Loss=0.0144 | F1=0.5295
New best F1 = 0.5295
Epoch 5 | Loss=0.0136 | F1=0.5555
New best F1 = 0.5555
Epoch 6 | Loss=0.0131 | F1=0.5729
New best F1 = 0.5729
Added 353 pseudo-labeled examples
Epoch 7 | Loss=0.0124 | F1=0.5868
New best F1 = 0.5868
Epoch 8 | Loss=0.0121 | F1=0.5994
New best F1 = 0.5994
Epoch 9 | Loss=0.0118 | F1=0.6076
New best F1 = 0.6076
Added 485 pseudo-labeled examples
Epoch 10 | Loss=0.0113 | F1=0.6156
New best F1 = 0.6156
Epoch 11 | Loss=0.0111 | F1=0.6228
New best F1 = 0.6228
Epoch 12 | Loss=0.0109 | F1=0.6299
New best F1 = 0.6299
Added 481 pseudo-labeled examples
Epoch 13 | Loss=0.0105 | F1=0.6353
New best F1 = 0.6353
Epoch 14 | Loss=0.0104 | F1=0.6430
New best F1 = 0.6430
Epoch 15 | Loss=0.0102 | F1=0.6499
New best F1 = 0.6499
Added 410 pseudo-labeled examples

In [93]:
import csv
import numpy as np
from pathlib import Path

print("\n📝 Generating submission...")

teacher.eval()
X_test = X_test.to(device)

def select_k(prob, min_k=2, max_k=3):
    idx = np.argsort(prob)[::-1]    # sorted descending

    # Always take the best 3 candidates
    top3 = idx[:max_k]

    # If the 3rd is much weaker → keep only 2
    if prob[top3[2]] < 0.25 * prob[top3[1]]:
        return top3[:2]

    return top3

preds = []

with torch.no_grad():
    for start in tqdm(range(0, len(X_test), 64)):
        batch = X_test[start:start+64]

        logits = teacher(batch, use_dropout=False)
        probs = torch.sigmoid(logits).cpu().numpy()

        for p in probs:
            labels = select_k(p)
            preds.append([str(x) for x in labels])

# ==========================================================
# SAVE CSV
# ==========================================================

OUT_DIR = Path("Submission")
OUT_DIR.mkdir(exist_ok=True)
OUT_PATH = OUT_DIR / "submission_self.csv"

with open(OUT_PATH, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["id","label"])
    for pid, labels in zip(test_ids, preds):
        w.writerow([pid, ",".join(labels)])

print(f"🎉 Submission saved → {OUT_PATH}")



📝 Generating submission...


100%|██████████| 308/308 [00:00<00:00, 739.25it/s]

🎉 Submission saved → Submission\submission_self.csv
